# Соединения Солнца и Астрономических Объектов

## Список соединений на один день (база данных V50)

In [1]:
from suneph import alex_se
from compute import *
from utils import parse_date, parse_coord
import time

from vdb import process_vdb, handle_date

day = int(time.strftime("%d"))
month = int(time.strftime("%m"))
year = int('20' + time.strftime('%y'))

records = alex_se(loadfile = 'sun_ephemeris.npz')

catalogs = ['V/50', 'VII/239A/icpos']
        
handle_date([day, month, year], records, catalogs[1])


Catalogue VII/239A/icpos: mean position of NGC/IC objects
 13: 0:54.92.. 13: 8:16.81 -22:15: 6.4 ..+ 8:30:13.7 
Date        Time        NGC/IC       Cat          RAJ2000      DEJ2000      Dec distance
11/10/2015  10:11:45.61         4136            I  13: 4:19.04 - 7:38:57.5  + 0:50:56.8 
11/10/2015  14:10:24.36         4156            I  13: 4:55.66 - 7:56:48.0  + 1: 5: 1.51
11/10/2015   7:32:10.41         4173            I  13: 3:54.58 -11:30:20.6  + 4:44:50.98
11/10/2015   7:47:42.09         4176            I  13: 3:56.96 -11:29:50.7  + 4:44: 6.38
11/10/2015   1:39:20.04         4928            N  13: 3: 0.5  - 8: 5: 5.0  + 1:25: 9.63
11/10/2015   7:47:42.09         4933            N  13: 3:56.96 -11:29:50.2  + 4:44: 5.88
11/10/2015   7:32:10.41         4933            N  13: 3:54.58 -11:30:20.6  + 4:44:50.98
11/10/2015   8:15:25.81         4933            N  13: 4: 1.21 -11:29:26.8  + 4:43:16.22
11/10/2015   9:42: 0.54         4939            N  13: 4:14.48 -10:20:24.7  + 3:32:52.

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/vizier/core.py:685: UserWarning: VOTABLE parsing raised exception: 
  warnings.warn("VOTABLE parsing raised exception: {0}".format(ex))


## Список соединений на один месяц (база данных V50)

In [2]:
from suneph import alex_se
from compute import *
from utils import parse_date, parse_coord
import time

from vdb import process_vdb, handle_date

#day = int(time.strftime("%d"))
month = int(time.strftime("%m"))
year = int('20' + time.strftime('%y'))

records = alex_se(loadfile = 'sun_ephemeris.npz')

catalogs = ['V/50', 'VII/239A/icpos']
        
handle_date([ month, year], records, catalogs[0])

Catalogue V50: stars with m <= 6.5
 14:20:11.51.. 14:28:00.55 -29:37:22.10..+01:01:13.90
Date        Time        Name         HD           Vmag         RAJ2000      DEJ2000      Dec distance


/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/vizier/core.py:685: UserWarning: VOTABLE parsing raised exception: 
  warnings.warn("VOTABLE parsing raised exception: {0}".format(ex))


TypeError: comp_time() takes exactly 3 arguments (4 given)

In [2]:
from suneph import alex_se
from astroquery.simbad import Simbad
from compute import *
from utils import parse_date, parse_coord
import time

from vdb import process_vdb, handle_date


def g2j(r, g):
    rg = np.array([r, g])
    rg = np.reshape(rg, (-1,1))
    
    mat = np.array([[-0.60,  1.60],
                    [ 0.42,  0.58],
                    [ 1.15, -0.15]])
    bias = np.array([0.5, -0.03, 0.5])
    bias = np.reshape(bias, (-1,1))
    
    bvr = np.dot(mat, rg) + bias
    return bvr

def j2g(B, R):
    BR = np.array([B, R])
    BR = np.reshape(BR, (-1,1))
    
    mat = np.array([[-0.60,  1.60],
                    [ 1.15, -0.15]])
    bias = np.array([0.5, 0.5])
    bias = np.reshape(bias, (-1,1))
    
    invmat = np.linalg.inv(mat)
    
    rg = np.dot(invmat, BR - bias)
    return rg

def g2J(r, g):
    ret = -0.37*r + 1.37*g + 0.39
    return ret



year = 2016#int('20' + time.strftime('%y'))
dm = dpm(year)

vphi = 20.
cat = 'leda' 
phi= coord(vphi,0,0,1,'deg')

for month in range(1,13):
    days = range(1, dm[month-1]+1)
    fnm = './csv/' + str(month)+str(year)+'.csv'
    f = open(fnm, 'w+')
    f.write('Date,Time,NGC,LEDA,Messier,Name,Vmag,Dist,RA,DEC\n') #,Rmag,Bmag,rmag,gmag

    for i in xrange(len(days)):
        curdate = [days[i], month, year]
        [[ra1, dec1], [ra2, dec2]] = ephsun(curdate)

        if dec1.f2s() < dec2.f2s():
            c = dec1
            d = dec2
        else:
            c = dec2
            d = dec1

        ra_interest = [ra1.f2hd(), ra2.f2hd()]
        dec_interest = [(c - phi).f2hd(), (d + phi).f2hd()]

        customSimbad = Simbad()
        customSimbad.ROW_LIMIT = -1
        customSimbad.add_votable_fields('flux(V)')#, flux(B), flux(R), flux(r), flux(g)')
        
        query = "cat in ('" + cat + "')" + ' & '
        query += 'rah >= '+ str(ra_interest[0])  + ' & ' + 'rah < ' + str(ra_interest[1])+ ' & '
        query += 'dec >= '+ str(dec_interest[0]) + ' & ' + 'dec < ' + str(dec_interest[1])

        result = customSimbad.query_criteria(query)
        
        if result is not None:
            ll = len(result)
            result.sort('RA')
        else:
            continue

        for i in xrange(ll):
            row = result[i]
            [ra, dec] = [row['RA'], row['DEC']]

            ra = parse_coord( ra, 'ra', ' ')
            dec = parse_coord( dec, 'deg', ' ')
            try:
                t = comp_time(ra1, ra2, ra)
            except:
                print ra1, ra2, ra, row['MAIN_ID']
                continue
                
            tmp = str(curdate[0])
            if len(tmp) <  2:
                tmp = ' ' + tmp
            date = '' + tmp + '/'
            tmp = str(curdate[1])
            if len(tmp) < 2:
                tmp = ' ' + tmp
            date = date + tmp + '/' + str(curdate[2])
            dist = comp_dist(curdate[0], curdate[1], t, dec)

            hdist = abs(dist.f2s() / 3600.)
                            
            printer = '' + date + ',' + t.__str__(rpar = 0) 
            
            Vmag = row['FLUX_V']
            #Bmag = row['FLUX_B']
            #Rmag = row['FLUX_R']
            #rmag = row['FLUX_r']
            #gmag = row['FLUX_g']

            if str(Vmag) == '--':
                continue
                if (str(rmag) != '--') and (str(gmag) != '--'):
                    r = float(rmag)
                    g = float(gmag)
                    bvr = g2j(r, g)
                    bvr = bvr.flatten()
                    Vmag = bvr[1]
                else:
                    if (str(Bmag) != '--') and (str(Rmag) != '--'):
                        B = float(Bmag)
                        R = float(Rmag)
                        rg = j2g(B, R)
                        rg = rg.flatten()
                        bvr = g2j(rg[0], rg[1])
                        bvr = bvr.flatten()
                        Vmag = bvr[1]
                    else:
                        continue
            
            hdist = abs(dist.f2s() / 3600.)
            if (cat == 'leda'):
                if (str(Vmag)!='--'):
                    if (Vmag >= 20.):
                        continue
                        if (Vmag < 4):
                            if hdist >= 15:
                                continue
                        elif (Vmag < 5):
                            if hdist >= 14:
                                continue
                        elif (Vmag < 6):
                            if hdist >= 13:
                                continue
                        elif (Vmag < 7):
                            if hdist >= 12:
                                continue
                        elif (Vmag < 8):
                            if hdist >= 8:
                                continue
                        elif (Vmag < 9):
                            if hdist >= 4:
                                continue
                        elif (Vmag < 11):
                            if hdist > 2:
                                continue
                        else:
                            continue
                else:
                    continue
            
            alt_names = Simbad.query_objectids(row['MAIN_ID'])
            stnms = ''
            ngc = ''
            mes = ''
            leda = ''
            for an in alt_names:
                if an['ID'].startswith('NGC '):
                    if len(ngc) > 0:
                        ngc = ngc + ' / '
                    ngc = ngc + an['ID']
                if an['ID'].startswith('NAME '):
                    if len(stnms) > 0:
                        stnms = stnms + ' / '
                    tmp = an['ID'].replace("NAME", '')
                    stnms = stnms + tmp
                if an['ID'].startswith('M '):
                    mes += an['ID']
                if an['ID'].startswith('LEDA '):
                    if len(leda) > 0:
                        leda = leda + ' / '
                    leda += an['ID']
            stri = [ngc, leda, mes, stnms, str(Vmag)]#, str(Rmag), str(Bmag), str(rmag), str(gmag)]                           
            stri = stri +  [str(dist), str(ra), str(dec)]

            for x in stri:
                printer = printer + ',' + x
            f.writelines(printer+'\n')
        f.write('\n')
    f.close()
    print "Month", month, "is ready!"
print "Check it!"

Month 1 is ready!
Month 2 is ready!
Month 3 is ready!
Month 4 is ready!
Month 5 is ready!
Month 6 is ready!
Month 7 is ready!
Month 8 is ready!
 10:59:07.89  11:02:44.18  10:59:00.00 MCG-01-28-012
 12:14:33.18  12:18:09.51  12:14:00.00 LEDA   39126
 12:18:09.51  12:21:46.07  12:18:00.00 LEDA   39489
Month 9 is ready!
 13:05:29.35  13:09:10.54  13:05:00.00 MCG-01-33-083
Month 10 is ready!
 14:32:54.15  14:36:51.00  14:36:51.00 LEDA  119811
Month 11 is ready!
Month 12 is ready!
Check it!


/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('leda') & rah >= 23.9675444444 & rah < 0.0282916666667 & dec >= -20.2114444444 & dec < 20.1836111111': No object found
  (error.line, error.msg))


In [4]:
from suneph import alex_se
from astroquery.simbad import Simbad
from compute import *
from utils import parse_date, parse_coord
import time

from vdb import process_vdb, handle_date

#day = int(time.strftime("%d"))
month = 1#int(time.strftime("%m"))
year = 2016#int('20' + time.strftime('%y'))
dm = dpm(year)

vphi = 20.
cat = 'ocl' # 'GCl
phi= coord(vphi,0,0,1,'deg')
#fields = ['ID', 'RAJ2000', 'DEJ2000', 'Name']

Sq = Simbad()
Sq.add_votable_fields('id(ngc)')
Sqm = Simbad()
Sqm.add_votable_fields('id(m)')
for month in range(1,13):
    days = range(1, dm[month-1]+1)
    fnm = './csv/' + str(month)+str(year)+'.csv'
    f = open(fnm, 'w+')
    if cat == 'gcl':
        f.write('Date,Time,NGC,GCl,Messier,Name,Dist,RA,DEC\n')
    elif cat == 'ocl':
        f.write('Date,Time,NGC,OCl,Messier,Name,Dist,RA,DEC\n')


    for i in xrange(len(days)):
        curdate = [days[i], month, year]
        [[ra1, dec1], [ra2, dec2]] = ephsun(curdate)

        if dec1.f2s() < dec2.f2s():
            c = dec1
            d = dec2
        else:
            c = dec2
            d = dec1

        ra_interest = [ra1.f2hd(), ra2.f2hd()]
        dec_interest = [(c - phi).f2hd(), (d + phi).f2hd()]

        #fields = ['RA', 'DEC', 'flux(V)']
        #v = Vizier(columns = fields, catalog = cat)
        #v.ROW_LIMIT = -1
        #result = v.query_constraints(catalog = cat, RAJ2000 = ra_interest, DEJ2000 = dec_interest)

        customSimbad = Simbad()#columns = fields)
        if cat == 'gcl':
            customSimbad.add_votable_fields('id(gcl)')
            #customSimbad.add_votable_fields('id(ngc)')
        elif cat == 'ocl':
            customSimbad.add_votable_fields( 'id(ocl)')
        query = "cat in ('" + cat + "')" + ' & '
        query += 'rah > '+ str(ra_interest[0])  + ' & ' + 'rah < ' + str(ra_interest[1])+ ' & '
        query += 'dec > '+ str(dec_interest[0]) + ' & ' + 'dec < ' + str(dec_interest[1])

    
        customSimbad.ROW_LIMIT = -1


        result = customSimbad.query_criteria(query)
        
        if result is not None:
            ll = len(result)
            result.sort('RA')
        else:
            continue

        for i in xrange(ll):
            row = result[i]
            #row = result[0][i].as_void()
            #row = list(row)
            #row = row[:-2]
            [ra, dec] = [row['RA'], row['DEC']]

            ra = parse_coord( ra, 'ra', ' ')
            dec = parse_coord( dec, 'deg', ' ')
            try:
                t = comp_time(ra1, ra2, ra)
            except:
                print ra1, ra2, ra, row['MAIN_ID']
                continue
                
            tmp = str(curdate[0])
            if len(tmp) <  2:
                tmp = ' ' + tmp
            date = '' + tmp + '/'
            tmp = str(curdate[1])
            if len(tmp) < 2:
                tmp = ' ' + tmp
            date = date + tmp + '/' + str(curdate[2])
            dist = comp_dist(curdate[0], curdate[1], t, dec)

            hdist = abs(dist.f2s() / 3600.)
                            
            printer = '' + date + ',' + t.__str__(rpar = 0) 
            
            name = row['MAIN_ID']
            locres = Sq.query_object(name)
            locresm = Sqm.query_object(name)
            stri = [locres['ID_ngc']]
            if name.startswith('NAME '):
                name = name.replace("NAME ", '')
            else:
                name = ''
            mescod = locresm['ID_m']
            if cat == 'gcl':
                stri = stri + [row['ID_gcl']]
            elif cat == 'ocl':
                stri = stri + [row['ID_ocl']]

                               
            stri = stri +  [mescod, name, str(dist), str(ra), str(dec)]

            for x in stri:
                printer = printer + ',' + x
            f.writelines(printer+'\n')
            #print printer
        f.write('\n')
    f.close()
    print "Month", month, "is ready!"
print "Check it!"

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 18.91965 & rah < 18.9930027778 & dec > -42.8313333333 & dec < -2.73086111111': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 18.9930027778 & rah < 19.0662444444 & dec > -42.7308611111 & dec < -2.62286111111': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 19.0662444444 & rah < 19.1393722222 & dec > -42.6228611

Month 1 is ready!
Month

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 22.7367388889 & rah < 22.7993055556 & dec > -28.0130833333 & dec < 12.3656388889': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 22.7993055556 & rah < 22.8617361111 & dec > -27.6343611111 & dec < 12.7462222222': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 22.8617361111 & rah < 22.9240333333 & dec > -27.2537

 2 is ready!
Month

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 0.634963888889 & rah < 0.695688888889 & dec > -15.8971944444 & dec < 24.4895': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 0.695688888889 & rah < 0.75645 & dec > -15.5105 & dec < 24.8748888889': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 0.75645 & rah < 0.81725 & dec > -15.1251111111 & dec < 25.258833333

...

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 14.4174583333 & rah < 14.4828055556 & dec > -34.7 & dec < 5.619': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 14.4828055556 & rah < 14.548375 & dec > -35.0150277778 & dec < 5.3': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 14.548375 & rah < 14.6141666667 & dec > -35.3259722222 & dec < 4.98497222222': No o

 10 is ready!
Month

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 16.3334555556 & rah < 16.4050916667 & dec > -41.6163888889 & dec < -1.44947222222': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 16.4769138889 & rah < 16.5489138889 & dec > -41.9295277778 & dec < -1.77641666667': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 16.5489138889 & rah < 16.6210861111 & dec > -42.07

 11 is ready!
Month 12 is ready!
Check it!


/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('ocl') & rah > 18.4588722222 & rah < 18.5327472222 & dec > -43.2861388889 & dec < -3.23366666667': No object found
  (error.line, error.msg))


In [23]:
## NEBULAE


from suneph import alex_se
from astroquery.simbad import Simbad
from compute import *
from utils import parse_date, parse_coord
import time

from vdb import process_vdb, handle_date

#day = int(time.strftime("%d"))
month = 1#int(time.strftime("%m"))
year = 2016#int('20' + time.strftime('%y'))
dm = dpm(year)

vphi = 25.
# snr = supernova remnants
# pn = planetary nebulae
# DNe = dark nebulae
# rne = reflection nebulae
# hh =  Herbig-Haro Object
# HII = HII (ionized) region
# lydnx bright nebula
cat = 'rne' 

if cat == 'snr':
    vphi = 20.
if cat == 'pn':
    vphi = 5.
if cat == 'DNe':
    vphi = 20.
if cat == 'lbn':
    vphi = 18.
if cat =='hh':
    vpi = 3.

phi= coord(vphi,0,0,1,'deg')
#fields = ['ID', 'RAJ2000', 'DEJ2000', 'Name']

Sq = Simbad()
Sq.add_votable_fields('id(ngc)')
Sqm = Simbad()
Sqm.add_votable_fields('id(m)')
Sqbarn = Simbad()
Sqbarn.add_votable_fields('id(barnard)')
Sqldn = Simbad()
Sqldn.add_votable_fields('id(ldn)')
for month in range(1,13):
    days = range(1, dm[month-1]+1)
    fnm = './csv/' + str(month)+str(year)+'.csv'
    f = open(fnm, 'w+')
    if cat == 'snr':
        f.write("Super Nova Remnants\n")
        f.write('Date,Time,NGC,SNR,Messier,Name,Dist,RA,DEC\n')
    elif cat == 'pn':
        f.write("Planetary Nebulae\n")
        f.write('Date,Time,NGC,PN,Messier,Name,Dist,RA,DEC\n')
    elif cat == 'DNe':
        f.write("Dark Nebulae\n")
        f.write('Date,Time,NGC,Barnard,LDN,Messier,Name,Dist,RA,DEC\n')
    elif cat == 'rne':
        f.write("Reflection Nebulae\n")
        f.write('Date,Time,NGC,PN,Messier,Name,Dist,RA,DEC\n')
    elif cat == 'hh':
        f.write("Herbig-Haro Object\n")
        f.write('Date,Time,NGC,HH,Messier,Name,Dist,RA,DEC\n')
    elif cat == 'lbn':
        f.write("Bright Nebulae\n")
        f.write('Date,Time,NGC,LBN,Messier,Name,Dist,RA,DEC\n')
    


    for i in xrange(len(days)):
        curdate = [days[i], month, year]
        [[ra1, dec1], [ra2, dec2]] = ephsun(curdate)

        if dec1.f2s() < dec2.f2s():
            c = dec1
            d = dec2
        else:
            c = dec2
            d = dec1

        ra_interest = [ra1.f2hd(), ra2.f2hd()]
        dec_interest = [(c - phi).f2hd(), (d + phi).f2hd()]

        customSimbad = Simbad()
            
        if cat == 'snr':
            customSimbad.add_votable_fields('id(snr)')
        elif cat == 'pn':
            customSimbad.add_votable_fields('id(pn)')
        elif cat == 'DNe':
            customSimbad.add_votable_fields('id(dne)')
        elif cat == 'rne':
            customSimbad.add_votable_fields('id(rne)')
        elif cat == 'hh':
            customSimbad.add_votable_fields('id(hh)')
        elif cat == 'lbn':
            customSimbad.add_votable_fields('id(lbn)')
            
        query = "cat in ('" + cat + "')" + ' & '
        query += 'rah >= '+ str(ra_interest[0])  + ' & ' + 'rah < ' + str(ra_interest[1])+ ' & '
        query += 'dec > '+ str(dec_interest[0]) + ' & ' + 'dec < ' + str(dec_interest[1])

    
        customSimbad.ROW_LIMIT = -1
        result = customSimbad.query_criteria(query)
        
        if result is not None:
            ll = len(result)
            result.sort('RA')
        else:
            continue

        for i in xrange(ll):
            row = result[i]
            [ra, dec] = [row['RA'], row['DEC']]

            ra = parse_coord( ra, 'ra', ' ')
            dec = parse_coord( dec, 'deg', ' ')
            try:
                t = comp_time(ra1, ra2, ra)
            except:
                print ra1, ra2, ra, row['MAIN_ID']
                continue
                
            tmp = str(curdate[0])
            if len(tmp) <  2:
                tmp = ' ' + tmp
            date = '' + tmp + '/'
            tmp = str(curdate[1])
            if len(tmp) < 2:
                tmp = ' ' + tmp
            date = date + tmp + '/' + str(curdate[2])
            dist = comp_dist(curdate[0], curdate[1], t, dec)

            hdist = abs(dist.f2s() / 3600.)
                            
            printer = '' + date + ',' + t.__str__(rpar = 0) 
            
            name = row['MAIN_ID']
            locres = Sq.query_object(name)
            locresm = Sqm.query_object(name)
            stri = [locres['ID_ngc']]
            ldnid = Sqldn.query_object(name)
            if name.startswith('NAME '):
                name = name.replace("NAME ", '')
            mescod = locresm['ID_m']
            if cat == 'snr':
                stri = stri + [row['ID_snr']]
            elif cat == 'pn':
                stri = stri + [row['ID_pn']]
            elif cat == 'DNe':
                bid = Sqbarn.query_object(name)
                ldnid = Sqldn.query_object(name)
                stri = stri + [bid['ID_barnard'], ldnid['ID_ldn']]
            elif cat == 'rne':
                stri = stri + [row['ID_rne']]
            elif cat == 'hh':
                stri = stri + [row['ID_hh']]
            elif cat == 'lbn':
                try:
                    stri = stri + [ldnid['ID_lbn']]
                except:
                    stri = stri + [ldnid['MAIN_ID']]

                               
            stri = stri +  [mescod, name, str(dist), str(ra), str(dec)]

            for x in stri:
                printer = printer + ',' + x
            f.writelines(printer+'\n')
        f.write('\n')
    f.close()
    print "Month", month, "is ready!"
print "Check it!"

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('rne') & rah >= 18.6990388889 & rah < 18.7726611111 & dec > -48.0871666667 & dec < 1.99047222222': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('rne') & rah >= 18.7726611111 & rah < 18.8462 & dec > -48.0095277778 & dec < 2.07575': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('rne') & rah >= 18.8462 & rah < 18.91965 & dec > -47.92425 & dec < 2.16866666

Month 1 is ready!
Month

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('rne') & rah >= 22.7993055556 & rah < 22.8617361111 & dec > -32.6343611111 & dec < 17.7462222222': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('rne') & rah >= 22.8617361111 & rah < 22.9240333333 & dec > -32.2537777778 & dec < 18.1285555556': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('rne') & rah >= 22.9240333333 & rah < 22.9862055556 & dec > -31.8

 2 is ready!
Month

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('rne') & rah >= 0.634963888889 & rah < 0.695688888889 & dec > -20.8971944444 & dec < 29.4895': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('rne') & rah >= 0.695688888889 & rah < 0.75645 & dec > -20.5105 & dec < 29.8748888889': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('rne') & rah >= 0.75645 & rah < 0.81725 & dec > -20.1251111111 & dec < 30.258833

...

 9 is ready!
Month

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('rne') & rah >= 14.3523305556 & rah < 14.4174583333 & dec > -39.381 & dec < 10.9418333333': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('rne') & rah >= 14.4174583333 & rah < 14.4828055556 & dec > -39.7 & dec < 10.619': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('rne') & rah >= 14.4828055556 & rah < 14.548375 & dec > -40.0150277778 & dec < 10.3': No

 10 is ready!
Month

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('rne') & rah >= 16.4050916667 & rah < 16.4769138889 & dec > -46.7764166667 & dec < 3.38361111111': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('rne') & rah >= 16.4769138889 & rah < 16.5489138889 & dec > -46.9295277778 & dec < 3.22358333333': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('rne') & rah >= 16.5489138889 & rah < 16.6210861111 & dec > -47.0

 11 is ready!
Month 12 is ready!
Check it!


In [18]:
row

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,ID_lbn
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,
object,unicode13,unicode13,int16,int16,float32,float32,int16,unicode1,unicode1,object,object
LBN 027.91-17.50,19 46,-12.2,2,2,1080000.000,1080000.000,90,E,,,LBN 027.91-17.50


In [47]:
#customSimbad.add_votable_fields("NAME")
#result = customSimbad.query_object('pal 4')
#result
#customSimbad.list_votable_fields()
name#customSimbad.add_votable_fields('id(gcl)')
Sq.query_object(name)

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,ID_ngc
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,
object,unicode13,unicode13,int16,int16,float32,float32,int16,unicode1,unicode1,object,object
NGC 5634,14 29 37.28,-05 58 35.1,6,6,--,--,0,D,,2006MNRAS.365.1357D,NGC 5634


In [1]:
import ephem
import numpy as np
from utils import parse_coord

def dpm(year):
    ds = np.array([30]*12)
    ds[ :7:2] += 1
    ds[7: :2] += 1
    if year % 4 == 0:
        ds[1] = 29
    else:
        ds[1] = 28 
    return ds


def prev_date(date):
    ds = dpm(date[2])
    
    if date[0] == 1:
        if date[1] == 1:
            nd = [31, 12, date[2] - 1]
        else:
            nd = [ds[date[1]-2], date[1]-1, date[2]]
    else:
        nd = [date[0]-1, date[1], date[2]]
    return nd

def ephsun(date):
    # Moscow coordinates
    (lat, lon) = (55.7522200, 37.6155600) 

    date2 = prev_date(date)
    
    time = '21:00' #
    sdate1 = str(date2[2]) + '/' + str(date2[1]) + '/' + str(date2[0]) + ' '
    sdate2 = str(date[2]) + '/' + str(date[1]) + '/' + str(date[0]) + ' '
    sun = ephem.Sun()
    #gatech = ephem.Observer()
    #gatech.lon, gatech.lat = str(lon), str(lat)
    
    gatech = sdate1 + time
    #gatech.date = sdate1 + time

    sun.compute(gatech)
    sun1 = [parse_coord(str(sun.a_ra), 'ra'), parse_coord(str(sun.a_dec), 'deg')]
    
    gatech = sdate2 + time #.date
    #gatech.date = sdate2 + time

    sun.compute(gatech)
    sun2 = [parse_coord(str(sun.a_ra), 'ra'), parse_coord(str(sun.a_dec), 'deg')]
    return sun1, sun2

date = [1, 12, 2014]
print prev_date(date)
print ephsun(date)    

[30, 11, 2014]
([ra:   16:26:30.69, deg:  -21:41:58.70], [ra:   16:30:49.42, deg:  -21:51:21.90])


In [24]:
from astroquery.simbad import Simbad
from compute import *
from utils import parse_date, parse_coord
import time

#from vdb import process_vdb, handle_date

#day = int(time.strftime("%d"))
month = 1#int(time.strftime("%m"))
year = 2016#int('20' + time.strftime('%y'))
dm = dpm(year)

#cat = 'hip' #aco, hip, ngc, qso 
cat = 'qso'
if cat == 'hip':
    vphi = 17
elif cat == 'qso':
    vphi = 30
elif cat == 'aco':
    vphi = 30
elif cat == 'ngc':
    vphi = 25
else:
    vphi = 15
phi= coord(vphi,0,0,1,'deg')

for month in range(1,13):
    days = range(1, dm[month-1]+1)
    fnm = './csv/' + str(month)+str(year)+'.csv'
    f = open(fnm, 'w+')
    if cat == 'hip':
        f.write('Date,Time,HIP,HD,B/F,Name,Dist,Vmag,RA,DEC\n')
    elif cat == 'ngc':
        f.write('Date,Time,NGC,Messier,Name,Dist,RA,DEC\n')
    else:
        f.write('Date,Time,Code,Dist,RA,DEC\n')


    for i in xrange(len(days)):
        curdate = [days[i], month, year]
        [[ra1, dec1], [ra2, dec2]] = ephsun(curdate)

        if dec1.f2s() < dec2.f2s():
            c = dec1
            d = dec2
        else:
            c = dec2
            d = dec1

        ra_interest = [ra1.f2hd(), ra2.f2hd()]
        dec_interest = [(c - phi).f2hd(), (d + phi).f2hd()]

        #fields = ['RA', 'DEC', 'flux(V)']
        customSimbad = Simbad()#columns = fields)

        query = "cat in ('" + cat + "')" + ' & '
        query += 'rah >= '+ str(ra_interest[0])  + ' & ' + 'rah < ' + str(ra_interest[1])+ ' & '
        query += 'dec > '+ str(dec_interest[0]) + ' & ' + 'dec < ' + str(dec_interest[1])

        if cat == 'hip':
            customSimbad.add_votable_fields('flux(V)', 'id(hip)')
        elif cat == 'ngc':
            customSimbad.add_votable_fields('id(m)')
        customSimbad.ROW_LIMIT = -1


        result = customSimbad.query_criteria(query)


        if result is not None:
            ll = len(result)
            result.sort('RA')
        else:
            continue

        for i in xrange(ll):
            row = result[i]
            [ra, dec] = [row['RA'], row['DEC']]
            if cat == 'hip':
                vmag = row['FLUX_V']
            ra = parse_coord( ra, 'ra', ' ')
            dec = parse_coord( dec, 'deg', ' ')
            try:
                t = comp_time(ra1, ra2, ra)
            except:
                print ra1, ra2, ra, row['MAIN_ID']
                continue
                
            tmp = str(curdate[0])
            if len(tmp) <  2:
                tmp = ' ' + tmp
            date = '' + tmp + '/'
            tmp = str(curdate[1])
            if len(tmp) < 2:
                tmp = ' ' + tmp
            date = date + tmp + '/' + str(curdate[2])
            dist = comp_dist(curdate[0], curdate[1], t, dec)

            hdist = abs(dist.f2s() / 3600.)
            if (cat == 'hip'):
                if (str(vmag)!='--'):
                    if (vmag >= 3):
                        if (vmag < 4):
                            if hdist >= 15:
                                continue
                        elif (vmag < 5):
                            if hdist >= 14:
                                continue
                        elif (vmag < 6):
                            if hdist >= 13:
                                continue
                        elif (vmag < 7):
                            if hdist >= 12:
                                continue
                        elif (vmag < 8):
                            if hdist >= 8:
                                continue
                        elif (vmag < 9):
                            if hdist >= 4:
                                continue
                        elif (vmag < 11):
                            if hdist > 2:
                                continue
                        else: #(vmag <= 10):
                            if hdist > 1:
                                continue
                        #else:
                            #print "Strange! " + row['MAIN_ID'], vmag, hdist
                            #continue
                            
            printer = '' + date + ',' + t.__str__(rpar = 0) 

            stri = [row['MAIN_ID'], str(dist)]
            if cat == 'hip':
                alt_names = Simbad.query_objectids(row['MAIN_ID'])
                stnms = ''
                dl = ''
                hd = ''
                for an in alt_names:
                    if an['ID'].startswith('HD'):
                        if len(hd) > 0:
                            hd = hd + ' / '
                        hd = hd + an['ID']
                    if an['ID'].startswith('* '):
                        if len(dl) > 0:
                            dl = dl + ' / '
                        dl = dl + an['ID'].replace("*", "")
                    if an['ID'].startswith('NAME '):
                        if len(stnms) > 0:
                            stnms = stnms + ' / '
                        tmp = an['ID'].replace("NAME", '')
                        stnms = stnms + tmp
                stri = [row['ID_hip'], hd, dl, stnms, str(dist), str(vmag)]
            elif cat == 'ngc':
                mes = row['ID_m']
                if (len(mes) == 0) and (hdist > 15):
                    continue
                alt_names = Simbad.query_objectids(row['MAIN_ID'])
                stnms = ''
                ngc = ''
                for an in alt_names:
                    if an['ID'].startswith('NGC '):
                        if len(ngc) > 0:
                            ngc = ngc + ' / '
                        ngc = ngc + an['ID']
                    if an['ID'].startswith('NAME '):
                        if len(stnms) > 0:
                            stnms = stnms + ' / '
                        tmp = an['ID'].replace("NAME", '')
                        stnms = stnms + tmp    
                stri = [ngc, mes, stnms, str(dist)]
            stri += [str(ra), str(dec)]

            for x in stri:
                printer = printer + ',' + x
            f.writelines(printer+'\n')
            #print printer
        f.write('\n')
    f.close()
    print "Month", month, "is ready!"
print "Check it!"
            
    
# query_objectids()
# parse it by *, hd, hip, Name (the latter may be multiple, so concatenate it)

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('qso') & rah >= 18.6990388889 & rah < 18.7726611111 & dec > -53.0871666667 & dec < 6.99047222222': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('qso') & rah >= 18.7726611111 & rah < 18.8462 & dec > -53.0095277778 & dec < 7.07575': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('qso') & rah >= 18.91965 & rah < 18.9930027778 & dec > -52.8313333333 & dec <

Month 1 is ready!
Month

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('qso') & rah >= 20.4969416667 & rah < 20.5663666667 & dec > -48.9905 & dec < 11.25825': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('qso') & rah >= 23.9675444444 & rah < 0.0282916666667 & dec > -30.2114444444 & dec < 30.1836111111': No object found
  (error.line, error.msg))


 2 is ready!
Month 3 is ready!
Month 4 is ready!
Month

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('qso') & rah >= 4.599675 & rah < 4.66796666667 & dec > -7.96686111111 & dec < 52.1665555556': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('qso') & rah >= 4.73636944444 & rah < 4.80487777778 & dec > -7.70644444444 & dec < 52.4140833333': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('qso') & rah >= 5.35592222222 & rah < 5.425075 & dec > -6.86008333333 

 5 is ready!
Month

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('qso') & rah >= 6.60205277778 & rah < 6.67104166667 & dec > -6.88555555556 & dec < 53.1773888889': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('qso') & rah >= 7.01486388889 & rah < 7.08335555556 & dec > -7.40344444444 & dec < 52.6993888889': No object found
  (error.line, error.msg))


 6 is ready!
Month 7 is ready!
Month 8 is ready!
Month 9 is ready!
Month 10 is ready!
Month

/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('qso') & rah >= 17.3506083333 & rah < 17.424175 & dec > -53.1996666667 & dec < 6.86488888889': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('qso') & rah >= 17.8670138889 & rah < 17.9409833333 & dec > -53.4345555556 & dec < 6.57558333333': No object found
  (error.line, error.msg))
/home/hariyuki/downloads/ap/lib/python2.7/site-packages/astroquery/simbad/core.py:853: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'cat in ('qso') & rah >= 18.0889861111 & rah < 18.1629972222 & dec > -53.43133

 11 is ready!
Month 12 is ready!
Check it!


In [1]:
import requests

url = 'http://mpc.cfa.harvard.edu/ws/search'

arg =  [['neo', 1], ['inclination_max', 2.0], ['spin_period_min', 5.0]]

params = {}
for i in range(1, len(arg)):
    params[arg[i][0]] = arg[i][1]

r = requests.post(url, params, auth = ('mpc_ws', 'mpc!!ws'))

print r.text
#N,i,w,a,e,M - orbital elements
#http://www.stjarnhimlen.se/comp/ppcomp.html

N = N_Epoch + 0.013967 * ( 2000.0 - Epoch ) + 3.82394E-5 * d

# ecc = eccentricity; M - mean anomaly (deg); E - eccentric anomaly
Eold = M + ecc*(180/np.pi) * np.sin(M) * ( 1.0 + ecc * np.cos(M) )
Enew = Eold
while Enew >= 0.05:
    Enew = Eold - ( Eold - ecc*(180/np.pi) * np.sin(Eold) - M ) / ( 1 - ecc * np.cos(Eold) )
    Eold = Enew
    # if not conv, ecc ~ 1 =>parabolic, not ell.

<?xml version="1.0" encoding="UTF-8"?>
<properties>
  <property>
    <absolute-magnitude>7.13</absolute-magnitude>
    <albedo-neowise>0.05</albedo-neowise>
    <albedo-neowise-unc>0.02</albedo-neowise-unc>
    <albedo-updated-at>2014-07-23T19:43:54Z</albedo-updated-at>
    <aphelion-distance>2.83</aphelion-distance>
    <apparition-end-date nil="true"/>
    <apparition-end-magn nil="true"/>
    <apparition-start-date nil="true"/>
    <apparition-start-magn nil="true"/>
    <arc-length>59066</arc-length>
    <argument-of-perihelion>182.37742</argument-of-perihelion>
    <ascending-node>211.179912</ascending-node>
    <b-minus-v>0.719</b-minus-v>
    <b-minus-v-source>M</b-minus-v-source>
    <binary-object nil="true"/>
    <color-ignore>CGB</color-ignore>
    <color-updated-at>2014-07-23T19:44:17Z</color-updated-at>
    <critical-list-numbered-object>false</critical-list-numbered-object>
    <delta-v>9.0</delta-v>
    <designation>0000019</designation>
    <diameter-neowise>223.0</diam


## Полезные сведения

## 1. Немного о вычислениях

В астрономии используется несколько видов координат. Один из них - геоцентрические; представьте поверхность Земли с нанесённой на неё разметкой в виде меридианов и параллелей, при этом последние удаляются в бесконечность, образуя т.н. небесную сферу. С точки зрения земного наблюдателя по небесной сфере движутся все астрономические объекты. Измеряя их положение в одно и то же фикисированное время суток на протяжении времени, примерно соответствующему одному земному году, можем заметить, что астрономические объекты 

Теперь вспомним, что Земля вращается вокруг Солнца под углом. 

### Вторая экваториальная система:

Основная плоскость - плоскость небесного экватора, в случае геоцентрической координатной системы совпадает с плоскостью Земного экватора.

Координаты второй экваториальной системы - (прямое восхождение, склонение) = (Ra, Dec).

$\textbf{Ra}$ (обозначается как $\alpha$) - англ. $\textbf{right ascension}$ - прямое восхождение, дуга небесного экватора от точки весеннего равноденствия до круга склонения светила, или угол между направлением на точку весеннего равноденствия и плоскостью круга склонения светила. Является эквивалентом долготы, отсчитывается от точки весеннего равноденствия (напомним, так называют точку, где Солнце пересекает небесный экватор в весеннее время года).

$\textbf{Dec}$ (обозначается как $\delta$) - англ. $\textbf{declination}$ - склонение, равняется угловому расстоянию на небесной сфере от плоскости небесного экватора до объекта; имеет знак ($+ \delta$ для объектов выше небесного экватора (север), $- \delta$ для объектов ниже (расположенных к югу). Является эквивалентом земной широты, отсчитывается от плоскости небесного экватора (в этой же плоскости лежит земной экватор).

Обе величины могут быть записаны в одном из двух форматов: в часовом $\textbf{h:m:s}$, где $h \in [0,24)$, $m, s \in [0, 60)$ (часы:минуты:секунды), либо угловом $\bf{a^o \, b' \, c''}$ (или $\textbf{a:b:c}$), $ a \in [0, 360)$ для $\alpha$ и $a \in [-90, 90]$ для $\delta$, $b,c \in [0, 60)$ (градусы:минуты:секунды).

### Приближение времени соединения Солнца и астрономического объекта

Считаем, что объект априори образует соединение с Солнцем (т.е., окзывается в некоторой малой окрестности Солнца с точки зрения наблюдателя). Для определения времени нам потребуются Ra ($\alpha$) координата Солнца и объекта. Обозначим их как $\alpha_{Sun}$ и $\alpha_{obj}$.

Здесь важно сделать ремарку о том, в каком формате доступны координаты Солнца. Таблицы, содержащие эти координаты для заданного объекта (а также и ряд других параметров) называют $\textbf{эфемеридами}$. Эфемериды вычисляются с учётом локального положения наблюдателя на поверхности Земли. Данный скрипт использует эфемериды Александровича, доступные по ссылке: http://hea.iki.rssi.ru/~nik/ak/sun.htm . Указанные эфемериды Солнца вычислены на каждый день для часового пояса UTC +03:00 (12:00 по Московскому зимнему времени). На момент написания данного текста эфемериды доступны с 1.12.2014 по 31.01.2016.

Таким образом, в нашем распоряжении оказываются координаты Солнца, вычисленные с шагом в одни сутки. Для приближённого вычисления времени соединения можно вычислить среднюю скорость движения Солнца по Ra-координате за 24 часа:
$$v_{\text{ср}} = \frac{\alpha_{Sun}^{[1]} - \alpha_{Sun}^{[0]}}{\Delta t} = \frac{\Delta \alpha_{Sun}}{\Delta t},$$
где $\Delta t = 24$ часа, $\alpha_{Sun}^{[0]}, \, \alpha_{Sun}^{[1]}$ - соседние измерения Ra-координаты Солнца.

Далее, из предположения о том, что соединения в указанный 24-часовой промежуток (от полудня первого дня до полудня дня второго) всё-таки произойдёт, вытекает, что в некоторый момент времени Ra-координаты Солнца и объекта совпадут. Что же, тогда не составляет никакого труда вычислить длину пути по $\alpha$-координате:
$$s = \alpha_{obj} - \alpha_{Sun}^{[0]},$$
и время соединения (в приближении равномерного движения по Ra-координате!) вычисляется по простой школьной формуле путём деления длины пути на скорость движения:
$$t_{appr.} = t_0 + \frac{s}{v_{\text{ср}}},$$
где индекс "appr." является сокращением от "approximate" (приближённый), а $t_0$ - время измерения $\alpha_{Sun}^{[0]}$.

### Вычисление удалённости положения объекта от Солнечной эклиптики

Здесь тоже всё относительно просто: необходимы $\delta$-координаты Солнца в момент соединения и объекта, из координаты Солнца вычитаем координату объекта, получаем расстояние, на котором объект отстоит от Солнца по данной координате:
$$\rho(\text{Sun}, \text{obj}) = \delta_{\text{Sun}} - \delta_{\text{obj}},$$
т.к. $\alpha$-координаты в момент соединения совпадают. Координаты $\delta_{\text{obj}}$ для астрономических объектов доступны из астрономических каталогов. С Dec-координатой для Слнца приходится немного сложнее.


Существуют формулы для приближённого вычисления склонения Солнца в зависимости от дня в году и времени суток. Здесь была использована следующая формула:
$$\delta \approx \frac{\pi}{180} \arcsin\left[ \sin\left(-23.44^o \frac{\pi}{180}\right) \cos\left(\frac{2 \pi}{365.24}(N+10) + 2 \cdot 0.0167 \cdot \sin\left(\frac{2 \pi}{365.24} (N-2)\right)\right) \right]$$

Параметр N изменяется от 0 до 365(366), при этом полагается изменяющимся непрерывно. Это даёт возможность учитывать суточное время. Так, для 12 часов 1.01 числа $N=0.5$.




## 2. Каталоги астрономических объектов

### Сервисы

Существует большое количество самых разных астрономических каталогов. Наиболее привычное для человека представление - табличное, однако часто это уже конечная форма представления данных. Для хранения чаще всего используют базы данных. Здесь приведём несколько популярных сервисов, где можно обращаться с запросами к самым разным каталогам (сохранённым в виде баз данных):

1) Vizier
http://vizier.u-strasbg.fr/viz-bin/VizieR

Представлено более 14000 каталогов. Принцип работы прост: сначала необходимо выбрать (найти) интересующий каталог, далее вести работу внутри выбранного каталога.

2) Simbad
http://simbad.u-strasbg.fr/simbad/

Данный сервис, наоборот, ориентирован на объекты. Использует информацию из многих источников; удобен для поиска информации по отдельно взятому объекту или объектам вне зависимости от баз данных. 

3) Minor Planets Center (MPC)
http://www.minorplanetcenter.com/

Здесь можно найти информацию о всех малых объектах вроде астероидов или комет.

### Классификация


1) Hipparcos Input Catalogue (HIC)

https://en.wikipedia.org/wiki/Hipparcos
Для звёзд принято использовать HIP-код.

2) Обозначения Флемстида (F-код)

https://ru.wikipedia.org/wiki/Обозначения_Флемстида

3)  Обозначения Байера (B-код)

https://ru.wikipedia.org/wiki/Обозначения_Байера

4) Новый общий каталог туманностей и звёздных скоплений (NGC)

https://en.wikipedia.org/wiki/New_General_Catalogue

5) Abell catalogue

https://en.wikipedia.org/wiki/Abell_catalogue

6) Messier object

https://en.wikipedia.org/wiki/Messier_object

7) Henry Draper Catalogue

https://en.wikipedia.org/wiki/Henry_Draper_Catalogue


### Каталоги

V50

V239A

## График функции приближённого склонения Солнца

[Nmin, Nmax] - отрезок числа дней, для которых будет построен график.

step - шаг по времени. По умолчанию выбрали в один час [step = 1 / 24]

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
    sun = []
    for i in xrange(len(records[0])):
        sun.append([parse_date(records[0][i]), parse_coord(records[1][i], 'ra'), parse_coord(records[2][i], 'deg')])
Nmin = 0
Nmax = 365*2
step = 1. / 24.

def sundec(N):
    tmp = np.sin(-23.44 * np.pi / 180)
    tmp *= np.cos(2*np.pi / 365.24 * ( N + 10) + 2*0.0167*np.sin(2*np.pi / 365.24*(N-2)))
    return np.arcsin(tmp)*180/np.pi

N = np.arange(Nmin, Nmax, step)
f = sundec(N)

plt.plot(N, f)
plt.xlabel('Day')
plt.ylabel('Delta')